# Apply TOPP-RA and send to cable robot

A template for creating a real-time version

It is reasonably likely this file will eventually turn into a .py file.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import src.io as io
import src.topp as topp
import src.util as util
from src.communicate import CableRobot
plt.rc('axes', unicode_minus=False)  # Glyph 8722

In [ ]:
strokes = io.load_log('logs/G.txt')
for stroke in strokes:
    print(stroke.shape)

In [ ]:
# Recenter
W, H = 2.92, 2.32
stroke = topp.Stroke(strokes[0][::5], clean=True)
center = np.mean([np.max(stroke.x, axis=0), np.min(stroke.x, axis=0)], axis=0)
newx = stroke.x - center + [W / 2, H / 2]
stroke = topp.Stroke(np.hstack((stroke.t.reshape(-1, 1), newx)),
                     clean=False)

In [ ]:
# Do TOPP-RA
stroke.spline_interp()
stroke.retime(vmax=0.3, amax=3)

ts, xs, xds, xdds = stroke.sample_retimed(dt=0.05)
print(f'# of spline segments: {stroke.retimed_path.cspl.c.shape[1]}')

In [ ]:
# Plot TOPP-RA results
# fig, axes = plt.subplots(3, 2, figsize=(5, 6))
fig, axes = plt.subplots(3, 2, figsize=(12, 5))
stroke.plot_xva(axes, markersize=1.5, linewidth=0.5)
fig.tight_layout()

In [ ]:
# Create html animation
HTML(stroke.create_html_anim('test1.html', bnds='cablerobot', markersize=2))
# HTML(stroke.create_html_anim('test1.html', bnds='auto'))

In [ ]:
PORT = '/dev/tty.usbmodem100994303'

In [ ]:
io.send_spline_serial(stroke.retimed_path.cspl, PORT)

In [ ]:
t, x, v, _ = stroke.sample_retimed(dt=0.1)
print('Ground Truth:')
print(np.hstack((t[:10].reshape(-1, 1), x[:10], v[:10])))
print('Read back from robot:')
with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,100', port=PORT) as robot:
    robot.send('t*')
    io.read_spline_serial(robot, num_lines=10)